In [31]:
import torch
import pandas as pd
import lmdb
import io
import pickle
import os
import json
import urllib
from transformers import CLIPTokenizer
from transformers import CLIPFeatureExtractor
from transformers import CLIPProcessor
from PIL import Image

In [27]:
dataset_path = '../e-ViL/data/esnlive_train.csv'
dataset = pd.read_csv(dataset_path)
text = dataset.iloc[0]['hypothesis']
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [46]:
out1 = tokenizer(text)

out2 = tokenizer(text,padding="max_length",
            max_length=68,
            truncation=True)
print(out1)
print(out2)
print(len(out1['input_ids']))
print(len(out2['input_ids']))

{'input_ids': [49406, 1237, 896, 1656, 47569, 320, 17859, 2183, 269, 49407], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [49406, 1237, 896, 1656, 47569, 320, 17859, 2183, 269, 49407, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
10
68


In [34]:
features_extract = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")
img_path = '../e-ViL/data/flickr30k_images/flickr30k_images/178045.jpg'
img = Image.open(img_path)

In [35]:
feats = features_extract(img)
feats['pixel_values']

[array([[[-0.5659947 , -0.87256163, -0.8433648 , ..., -0.4638057 ,
          -0.50760096, -0.43460885],
         [-0.6389868 , -0.9309554 , -1.0769397 , ..., -0.37621516,
          -0.4638057 , -0.31782144],
         [-0.7995695 , -0.96015227, -1.1645303 , ..., -0.405412  ,
          -0.50760096, -0.2740262 ],
         ...,
         [-0.14264035, -0.11344349, -0.06964822, ..., -0.2740262 ,
           0.03254076,  0.12013142],
         [-0.14264035, -0.08424664, -0.04045137, ..., -0.25942776,
          -0.11344349, -0.04045137],
         [-0.12804192, -0.0550498 , -0.02585294, ..., -0.21563247,
          -0.2302309 , -0.2740262 ]],
 
        [[-0.32635912, -0.6415222 , -0.6115067 , ..., -0.37138242,
          -0.43141347, -0.32635912],
         [-0.37138242, -0.6865455 , -0.83662325, ..., -0.2813358 ,
          -0.38639018, -0.22130474],
         [-0.5064523 , -0.6865455 , -0.8966543 , ..., -0.29634356,
          -0.4164057 , -0.16127367],
         ...,
         [ 0.01881953,  0.0338273

In [36]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [53]:
#processed_batch = processor(text=batch["question"], images=imgs_to_encode, padding=True, return_tensors="pt")
out1 = processor(text=text, images=img, padding=True, return_tensors="pt")
out2 = processor(text=text, images=img, return_tensors="pt",padding="max_length", max_length=68,truncation=True)
print(out1)
print(out2)
print('OUT1')
for key in out1:
    print('key: ',key,' shape:',out1[key].shape)
print('OUT2')
for key in out2:
    print('key: ',key,' shape:',out2[key].shape)

{'input_ids': tensor([[49406,  1237,   896,  1656, 47569,   320, 17859,  2183,   269, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'pixel_values': tensor([[[[-0.5660, -0.8726, -0.8434,  ..., -0.4638, -0.5076, -0.4346],
          [-0.6390, -0.9310, -1.0769,  ..., -0.3762, -0.4638, -0.3178],
          [-0.7996, -0.9602, -1.1645,  ..., -0.4054, -0.5076, -0.2740],
          ...,
          [-0.1426, -0.1134, -0.0696,  ..., -0.2740,  0.0325,  0.1201],
          [-0.1426, -0.0842, -0.0405,  ..., -0.2594, -0.1134, -0.0405],
          [-0.1280, -0.0550, -0.0259,  ..., -0.2156, -0.2302, -0.2740]],

         [[-0.3264, -0.6415, -0.6115,  ..., -0.3714, -0.4314, -0.3264],
          [-0.3714, -0.6865, -0.8366,  ..., -0.2813, -0.3864, -0.2213],
          [-0.5065, -0.6865, -0.8967,  ..., -0.2963, -0.4164, -0.1613],
          ...,
          [ 0.0188,  0.0338,  0.0638,  ..., -0.0712,  0.2439,  0.2740],
          [ 0.0338,  0.0488,  0.0638,  ...,  0.0188,  0.1989,  0.2439],
    

In [ ]:
def getSize(lmdb_file_name="./data/my_image_db"):
    lmdb_env = lmdb.open(lmdb_file_name, readonly=True)
    stats = lmdb_env.stat()
    info = lmdb_env.info()
    lmdb_env.close()
    dbSize = stats['psize'] * (stats['leaf_pages'] + stats['branch_pages'] + stats['overflow_pages'])
    return dbSize/1024/1024
getSize()

In [ ]:
class ImageDB():
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.feature_extract = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")
        data_path = '../e-ViL/data/'
        images_path ='flickr30k_images/flickr30k_images/'
        self.path = data_path+images_path
        self.images = [f for f in os.listdir(self.path)]
        self.images.sort()
        return
    
    def get_visual_features(self,img):
        return self.feature_extract(img)
    
    def write_to_lmdb(self,filename,map_size = 1000000000):#1GB
        k = 0
        env = lmdb.open(filename, map_size= map_size)
        for idx in range(len(img_list)):
            print(k)
            k+=1
            txn = env.begin(write=True)
            img = self.images[idx]
            img_path = self.path+img
            
            features = self.get_visual_features(img_path)
            
            item = {'features': features.numpy()}
            txn.put(key = img.encode(), value = pickle.dumps(item))
            # Commit changes through the commit() function 
            txn.commit()
            if(k%1000==0):
                torch.cuda.empty_cache()
        env.close()